# Lidar Processing Workflow
**Cara Piske, Graduate Program of Hydrologic Sciences, 2022; Advisor: Dr. Adrian Harpold**<br>
This code processes raw lidar point clouds in order to calculate snow depth using PDAL. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy estimates. Therefore, little interpolation occurs under-canopy. We follow these protocols in order to obtain a 1-m rasterized product (as opposed to the 3-m rasterized product provided by ASO on the NSIDC data portal). NCALM and WSI flights were obtained through OpenTopography.

Start by importing necessary files

In [1]:
# import necessary files
import numpy as np 
import matplotlib.pyplot as plt
import json # where we will save the json files to run a pipeline
import os # for file management
import subprocess # allows us to run command line commands
import pdal # lidar processing package
import rasterio
import pyproj
from osgeo import gdal

import time
# packages to copy files
from pathlib import Path
import shutil
import glob
# packages to extract wkt from polygon
import shapefile
import pygeoif
# for parallel processing
import concurrent.futures
from multiprocessing import Pool
# See lidar_functions.py
import lidar_functions
pdal_pipeline = 'C:\\Users\cpiske\.conda\envs\lidar\Lib\site-packages\pdal\pipeline.py'

Note that many functions are dependent on specific directory structures. See README

In [2]:
# # Set current working directory
# path = 'path/to/current/working/directory/'
# os.chdir(path)
# os.getcwd() # print to ensure we're in the right directory

In [3]:
# json_base_path = 'piske_processing/PDAL_workflow/JSON/' # set so that we can redefine json across operating systems

Applied

In [4]:
# Set current working directory
path = '/'
os.chdir(path)
os.getcwd() # print to ensure we're in the right directory

'G:\\'

In [5]:
json_base_path = 'piske_processing/PDAL_workflow/JSON/' # set so that we can redefine json across operating systems

# Pre-processing

## Info
Get to know lidar file... <br>
In general, running PDAL python bindings can be difficult, so an easy workaround is to initiate a terminal command and run via subprocess. This is what we'll do for most of the processing. 

In [ ]:
# # define input file
# input_lid = 'full/path/to/las/lidar.las'
# pdal_info_cmd = ['pdal','info',input_lid] # general info
# pdal_metadata_cmd = ['pdal','info',input_lid,'--metadata'] # full file metadata, including details crs
# subprocess.run(pdal_metadata_cmd)

Save results to a dictionary <br>
This will be useful below when renaming files based on specific metadata

In [ ]:
# pdal_info_results = subprocess.run(pdal_metadata_cmd, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
# pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # create dict with metadata info

## Retile
Retiling lidar files can help with memory issues and allow for parallelization, speeding up processing. Standard tiles come in 500-1500m side lengths. Choose depending on point density and spatial extent of files. Many NCALM files come in 1000x1000m already. Buffer zones are dependent on point density etc. and allow for redundant classifications to avoid the edge effect.

**Single Folder**

In [ ]:
# retile_input_path = 'path/to/input/lidar/folder/*.la*' # use * wildcard to identify all files that end in las or laz
# retile_output_path = 'path/to/output/lidar/folder/#.laz' # # will be replaced by a number
# onlyfiles = [f for f in os.listdir(retile_input_path) if os.path.isfile(os.path.join(retile_input_path, f))]
# for files in onlyfiles:
#     full_path = os.path.join(retile_input_path, files)
#     output_path = retile_output_path+'retile_#'+files[-4:]
#     retile_command = ['pdal', 'tile', full_path, output_path, '--length=1000','--buffer=50']
#     subprocess.run(retile_command)

**Multiple subdirectories**

In [ ]:
# all_folders = [x[0] for x in os.walk('folder1/folder2/SCB/')]
# # list indices of all folders that are called laz or las (i.e. 'folder1/folder2/SCB/flight1/laz/')
# index_pos_list = [ i for i in range(len(all_folders)) if (all_folders[i][-3:] == 'laz' or all_folders[i][-3:] == 'las') ]
# # create list of all folder paths that end with directory named 'laz' or 'las'
# lid_list = [all_folders[i] for i in index_pos_list]

# # for each of these folders...
# for lid_folders in lid_list:
#     # input wildcard string (ex. 'folder1/folder2/SCB/flight1/laz/*.laz')
#     input_file_wildcard = lid_folders + '/*.'+ lid_folders[-3:] # [-3:] allows us to use .las or .laz
#     # output pathname w/ wildcard (ex. 'folder1/folder2/SCB/flight1/retile/retile_#.laz')
#     output_path = lid_folders[:-3] + '/retile/retile_#.' + lid_folders[-3:]
#     # create pdal command
#     retile_cmd = ['pdal', 'tile', input_file_wildcard, output_path,'--length=1000','--buffer=50']
#     subprocess.run(retile_cmd)

## Rename
Many files come with inconsistent naming (including unsupported characters...) <br>
Rename all files to maintain consistency

In [ ]:
# # See lidar_functions.py
# # This is function .py file 
# # rename a file with the lower left x and y defined as the corner point of the bounding box (add resolution/2 to get the center point of the box) 
# # input - full lidar file path (i.e. 'lidar_files/filename.laz'), str
# # 
# def rename_llx_lly_b(full_path):
    
#     pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal info command
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dictionary
#     pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only pathname (i.e. 'folder1/folder2/SCB/flight1/lid_files/')
#     new_name = os.path.join(pathname, str(round(pdal_info_dict['metadata']['minx'])) +"_"+ str(round(pdal_info_dict['metadata']['miny']))+full_path[-4:])    
#     if os.path.exists(new_name):
#         lidar_folder = pathname
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
#         full_str = ','.join(full_paths)
#         num_occurences = full_str.count(new_name[:-4])
#         new_name_b = new_name[:-4]+'_'+str(num_occurences)+new_name[-4:]
#         os.rename(full_path, new_name_b)
#     else:
#         os.rename(full_path, new_name) # rename file

In [ ]:
# # See lidar_functions.py
# # rename a file with additional metadata at the beginning of the filename
# # flight organization, watershed, date of flight (i.e. ASO_ICB_20140423)
# # additional text will be taken from the flight directory name, could hardcode additional string instead of using folder name
# # input - full lidar file path (i.e. 'lidar/lidar_files/filename.laz'), str

# def add_str_to_filename2(full_path):
#     filename = os.path.basename(full_path) # isolate only filename (i.e 'filename.laz')
#     pathname = os.path.dirname(os.path.realpath(full_path)) # isolate only path name (i.e. 'lidar/lidar_files')
#     add_str = os.path.normpath(pathname) # split up the path name (i.e. full path)
#     add_str = [i for i in add_str.split(os.sep) if (i.startswith('ASO_') or i.startswith('NCALM_') or i.startswith('WSI_'))] # (i.e. 'ASO_SCB_2016')
#     rename = os.path.join(pathname, add_str[0] + '_'+ filename) # add string to full pathname 
#     os.rename(full_path, rename) # rename file

**Single file**

In [ ]:
# rename_llx_lly('filepath.laz')
# add_str_to_filename2('new_filepath.laz')

**Multiple Folders**

In [ ]:
# all_folders = [x[0] for x in os.walk('path/to/folder/')]
# for folders in all_folders:
#     # list indices of all folders that are called laz
#     index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
#     # save only those files 
#     retile_list = [all_folders[i] for i in index_pos_list]
#     for lidar_folder in retile_list:
#         rename_retiles(lidar_folder)

**Parallel Processing**

Single Folder

In [ ]:
# lidar_folder = 'path/to/lid/folder/'
# tic = time.perf_counter()
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#         executor.map(lidar_functions.rename_llx_lly, full_path) #running 10 times
# toc = time.perf_counter()

*Repeat with lidar_functions.add_str_to_filename...*

use if there are - in the output filename

In [ ]:
# # use if there are - in the output filename
# pathname = 'Piske_lidar/MRB/Merced_lidar/ASO/ASO_MRB_20210429/tindex/tiles/'
# for files in [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]:
#     target = files.replace("-","")
#     target = pathname + target
#     full_path = pathname+files
#     os.rename(full_path,target)

## Save tile boundaries
Save sqlite (shp) of tile boundaries of retiles <br>
Alter filepaths 

In [ ]:
# # see lidar_functions.py
# # save the tile index of a file to a new folder

# # input - full lidar/sqlite file patsh (i.e. 'lidar/lidar_files/filename.laz'), [str, str]
# def create_tindex2(input_path, output_path):
#     boundary_cmd = ['pdal', 'tindex', 'create', '--tindex', output_path, '--filespec', input_path, '-f', 'SQLite']
#     subprocess.run(boundary_cmd)

**Parallel Processing**

Single folder

In [ ]:
# # one folder
# tic = time.perf_counter()
# lidar_folder = 'path/to/lidar/files/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#         full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#         # change depending on directory formats
#         output_path = [os.path.join(os.path.basename(lidar_folder),'tindex' + s[:-3] + 'sqlite') for s in onlyfiles]
#         executor.map(create_tindex, full_path, output_path) #running 10 times
# toc = time.perf_counter()

all folders called "retile"

In [ ]:
# # rename all retiled files
# tic = time.perf_counter()
# all_folders = [x[0] for x in os.walk('path/to/folder/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# # save only those files 
# retile_list = [all_folders[i] for i in index_pos_list]
# for lidar_folder in retile_list:
#     if __name__ == '__main__':
#         with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#             onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#             full_path = [lidar_folder + '/' + s for s in onlyfiles]
#             output_path = [lidar_folder[:-6] + 'tindex/' + s[:-3] + 'sqlite' for s in onlyfiles]
#             executor.map(create_tindex, full_path, output_path) #running 10 times
# toc = time.perf_counter()

All files in ssd

In [ ]:
# tic = time.perf_counter()
# # list all folders in directory
# all_folders = [x[0] for x in os.walk('Piske_lidar/MRB/Merced_lidar/NCALM/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-3:] == 'laz' ]
# # save only those files 
# laz_list = [all_folders[i] for i in index_pos_list]
# for folders in laz_list:
#     pathname = folders + '/'
#     output_pathname = folders[:-3] + 'sqlite' + '/'
#     onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
#     for file in onlyfiles:
#         input_las = pathname + file
#         output_sqlite = output_pathname + file[:-4] + '.sqlite'
#         boundary_cmd = ['pdal', 'tindex', 'create', '--tindex', output_sqlite, '--filespec', input_las, '-f', 'SQLite']
#         subprocess.run(boundary_cmd)
# toc = time.perf_counter()

## Copy files
Save a copy of files that meet criteria into a new folder <br>
The goal is to decrease processing time for datum conversions in the MRB <br>
Change file extents depending on watershed

In [ ]:
# # see lidar_functions.py
# # copy file based on llx and lly into output filename. Note that this function relies on the assumption that files follow the structure 

# # input_las 'folder1/folder2/folder3a/filename.laz' and output_las 'folder1/folder2/folder3b/filename.laz'
# # if files are held in a different file structure, code must be changed to accomodate change
# # input - full path to a las file
# def copy_las_by_ext_ICB2(full_path):
#     pdal_info_command = ['pdal', 'info', full_path, '--metadata'] # set up pdal command
#     pdal_info_results = subprocess.run(pdal_info_command, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created, execute command
#     pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # save metadata to dict
#     minx = round(pdal_info_dict['metadata']['minx'])
#     miny = round(pdal_info_dict['metadata']['miny'])
#     if (minx <= 288000 and minx >= 265000):
#         if (miny >= 4165000 and miny<= 4180000):
#             input_las = full_path
#             output_las = os.path.join(os.path.dirname(os.path.realpath(os.path.dirname(os.path.realpath(full_path)))),'retile_ICB', os.path.basename(full_path))
#             pdal_copy_cmd = ['pdal','translate', input_las, output_las]
#             subprocess.run(pdal_copy_cmd)

**Parallel Processing**

All folders called "retile"

In [ ]:
# tic = time.perf_counter()
# all_folders = [x[0] for x in os.walk('path/to/lidar/flight/')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ]
# # save only those files 
# retile_list = [all_folders[i] for i in index_pos_list]
# for lidar_folder in retile_list:
#     if __name__ == '__main__':
#         with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#             onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#             full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
#             executor.map(copy_las_by_ext_ICB, full_path) #running 10 times
# toc = time.perf_counter()

### Copy tindex

In [ ]:
# all_folders = [x[0] for x in os.walk('Piske_lidar/MRB/Merced_lidar/ASO')]
# # list indices of all folders that are called laz
# index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-9:] == 'ICB_tiles' ]
# # save only those files 
# ICB_retile_list = [all_folders[i] for i in index_pos_list]
# for lidar_folder in ICB_retile_list:
#     onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
#     for files in onlyfiles:
#         tindex_full_path = os.path.dirname(lidar_folder) + '/tindex/tiles/'+files[:-3]+'sqlite'
#         if os.path.isfile(tindex_full_path):
#             target_full_path = os.path.dirname(lidar_folder) + '/tindex/ICB_tiles/'
#             shutil.copy2(tindex_full_path, target_full_path)
    

Applied

In [ ]:
tic = time.perf_counter()
lidar_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/'
output_path = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/ICB_tiles/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
output_paths = np.repeat(output_path, len(full_paths))
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        executor.map(lidar_functions.copy_lid_by_ext_ICB, full_paths, output_paths)
toc = time.perf_counter()

## Datum Conversions

### Horizontal Datum Conversion
From PDAL library: https://pdal.io/tutorial/grid-shift.html

In [ ]:
horizontal_datum_conversion = json_base_path+'horizontal_datum_conversion.json'

In [ ]:
# input_las = 'path/to/filename/filename.las'
# output_las = "path/to/filename/filename.las"

In [ ]:
# reader_dict = {"type":"readers.las",
#               "filename":input_reproj_hor}

# reproject_dict = {"type":"filters.reprojection",
#                   "in_srs":"+proj=utm +zone=10 +datum=WGS84 +units=m +no_defs",
#                   "out_srs":"EPSG:26910"} #"in_srs":"EPSG:8999"
# writer_dict = {"type":"writers.las",
#                "a_srs":"EPSG:26910",
#               #  "scale_x":0.00000001,
#               #  "scale_y":0.00000001,
#               # "offset_x":"auto",
#               # "offset_y":"auto",
#               "filename":output_reproj_hor}

# pipeline_list = [reader_dict, reproject_dict, writer_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(horizontal_datum_conversion, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

### Vertical Datum Conversion
using output of code above

In [ ]:
# # best option for now 
# input_las = 'path/to/filename.las'
# output_las = 'path/to/filename.las'
# gtx_path = 'path/to/fielname.gtx' # /cpiske/lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/gtx_links/core/geoid12b/g2012bu5.gtx
# reproj_fil_in_srs = '--filters.reprojection.in_srs=EPSG:26910+5703'
# reproj_fil_out_srs = '--filters.reprojection.out_srs=+init=EPSG:26910' + ' +geoidgrids='+gtx_path + ' +t_epoch=2010.0'
# writers_compression = '--writers.las.compression=true'
# writers_a_srs = '--writers.las.a_srs=+proj=utm +zone=10 +ellps=GRS80 +datum=NAD83 +units=m +no_defs' 

# Rasterize
Set up the rasterization pipeline which we will use throughout the workflow. <br>
We use two pipelines here, one which takes the mean of all points in a 1m pixel, using a 0.7m radius

In [ ]:
# name JSON file
rasterize_json_mean = json_base_path+'rasterize_mean.json'
rasterize_json_count = json_base_path+'rasterize_count.json'

In [ ]:
# create a pipeline and save to a json file 
reader_dict = {'type':'readers.las'}
writers_gdal= {"type": "writers.gdal",
              'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7'}#,
             #'window_size':3,
              #'default_srs':'EPSG:26910'}

pipeline_list = [reader_dict,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(rasterize_json_mean, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
# create a pipeline and save to a json file 
reader_dict = {'type':'readers.las'}
writers_gdal= {"type": "writers.gdal",
              'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7'}#,
             #'window_size':3,
              #'default_srs':'EPSG:26910'}

pipeline_list = [reader_dict,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(rasterize_json_mean, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# # Single Files
# reader = '--readers.las.filename='+'path/to/file/filename.las'
# writer = '--writers.gdal.filename='+'path/to/file/filename.tif'
# rasterize_command = ['pdal', 'pipeline', rasterize_json_mean, writer, reader]
# subprocess.run(rasterize_command)

**parallel processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/'
# output_path = 'path/to/tif/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_input_las = [input_path + '/' + s for s in onlyfiles]
#         full_output_tif = [output_path + '/' + s[:-4] + '.tif' for s in onlyfiles]
#         executor.map(rasterize_count, full_input_las, full_output_tif) 
# toc = time.perf_counter()

Applied

In [ ]:
#time = 3.2 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_las/'
output_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + s for s in onlyfiles]
        full_output_tif = [output_path + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(lidar_functions.rasterize_mean, full_input_las, full_output_tif) 
toc = time.perf_counter()

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/corrected_las/'
output_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/corrected_tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + s for s in onlyfiles]
        full_output_tif = [output_path + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(lidar_functions.rasterize_mean, full_input_las, full_output_tif) 
toc = time.perf_counter()

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered_tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_input_las = [input_path + s for s in onlyfiles]
        full_output_tif = [output_path + s[:-4] + '.tif' for s in onlyfiles]
        executor.map(lidar_functions.rasterize_mean, full_input_las, full_output_tif) 
toc = time.perf_counter()

# Ground Filter

## Assign Ground
Modified from [Chambers, Bradley (2017). Ground Filter Tutorial.](https://pdal.io/tutorial/ground-filters.html) <br>
See [SMRF filter documentation](https://pdal.io/stages/filters.smrf.html)

In [ ]:
# name JSON file
assign_ground_json = json_base_path +'assign_ground.json'

In [ ]:

# create a pipeline and save to a json file 

filter_dict_assign = {'type':'filters.assign', # set the value of a dimension for all points to a provided value that pass a range filter
                      "assignment":"Classification[:]=0"} #  single option has been provided that specifies the dimension, range, and value to assign. In this case, we are stating that we would like to apply a value of 0 to the Classification dimension for every point

filter_dict_elm = {'type':'filters.elm'} # identify low noise points that can adversely affect ground segmentation algorithms, automatically assigns value of 7 

filter_dict_outlier = {'type':'filters.outlier'} #  two methods of outlier detection at the moment: radius and statistical. Both aim to identify points that are isolated and likely arise from noise sources, classify values as 7
# classify ground points
filter_dict_smrf = {"type":"filters.smrf",
                    "ignore":"Classification[7:7]"}#, # ignore outliers
                    #"slope":0.3,
                    #"window":16,
                    #"threshold":0.15,
                    #"scalar":1.2}


pipeline_list = [filter_dict_assign,filter_dict_elm, filter_dict_outlier, filter_dict_smrf]
pipeline_dict = {'pipeline' : pipeline_list}
with open(assign_ground_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single File**

In [ ]:
# pdal_commands = ['pdal', 'translate', 'input_las.las', "output_las.las", '--json', assign_ground_json]
# subprocess.run(pdal_commands)

**All Files in Folder**

In [ ]:
# pathname = 'path/to/lidar/folder/'
# output_pathname = "path/to/output/lidar/folder/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', gf_json]
#     subprocess.run(pdal_commands)

## Filter by Ground
Filter classified las files based on [Las 1.4 Specifications](http://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf)<br>
Where Classification 2 = ground <br>
Classification 7 = noise

In [ ]:
gf_json = json_base_path+'ground_filter_preClassified.json' # define path to json files

filter_range = {"type":"filters.range", 
                "limits":"Classification[2:2]"}
pipeline_list = [filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(gf_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single File**

In [ ]:
# input_las = 'path/to/lid/file/filenam.las'
# output_las = 'path/to/lid/file/filename.las'

In [ ]:
# range_cmd = ['pdal', 'translate', input_las,  output_las, '--json',gf_json]
# subprocess.run(range_cmd)

**All Files in Directory**

In [ ]:
# #329.1668573822826s
# tic = time.perf_counter()
# input_path = 'path/to/input/directory'
# output_path = 'path/to/output/directory'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + '/' + s for s in onlyfiles]
#         output_path_full = [output_path + '/' + s for s in onlyfiles]
#         executor.map(lidar_functions.ground_filter_preClassified, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

## Create DEM
This pipeline combines aboves steps to avoid issues with merging rasters. We were seeing tile signatures when we merged all tif files after ground-filtering/rasterization, so here we combine these steps through merging the las and then rasterizing (importantly, we are not writing the merged las files to a new merged file, which defeatst the purpose of tiles). A weakness of this pipeline is that it doesn't allow for tile-level parallelization <br> see: https://pdal.io/pipeline.html

In [ ]:
DEM_json = json_base_path+'DEM_from_las.json' # define path to json files

Define the las files we want to create a DEM from.

In [ ]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/ground_filtered/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/testb.tif'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))] # make a list of all filenames in directory
input_list = [input_path + s for s in onlyfiles] # make a list of full filename paths in directory

Create the reader stages of the pipeline. Each file is read as an individual reader stage here, and we cheat here by copying the formatting of json files and creating a dictionary with the values as the correctly formatted inputs.

In [ ]:
filename_dict = {} # initiate an empty dict to hold the readers
tags = ['']*len(input_list) # initiate an empty list, size = number of files
filenames = ['']*len(input_list) # repeat
for i in range(len(input_list)): # for each file, create a dictionary element with the values matching json formatting for file merging
    filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
    tags[i] = 'A_'+str(i) # add a tag to the reader stage
    filenames[i] = filename_dict[list(filename_dict)[i]] # Add all values to a list

Define the filter and writer stages of the pipeline

In [ ]:
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7',
               'window_size':3,
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
#pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(DEM_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
pdal_cmd = ['pdal','pipeline', DEM_json]
subprocess.run(pdal_cmd)

# HAG and Noise Filter
Height above ground DEM (raster format). 

## Replace Z with HAG

In [ ]:
# define json path 
HAG_json = json_base_path + 'HAG_dem.json'

In [ ]:
# convert all z values to the height above ground 
target_dem = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/NCALM_2014_BE.tif'
filter_hag = {"type":"filters.hag_dem",
              "raster":target_dem, # full file path of target DEM (.tif)
              "zero_ground":"false"} # Do not assign 0 to ground classified points
filter_ferry = {"type":"filters.ferry",
                "dimensions":"HeightAboveGround=>Z"} # replace all Z dimensions with HAG instead of elevation
filter_range = {"type":"filters.range",
                "limits":"Z[-0.2:70]"} # apply a noise filter
pipeline_list = [filter_hag, filter_ferry,filter_range]#,filter_range]
pipeline_dict = {'pipeline' : pipeline_list}
with open(HAG_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# input_las = "path/to/full/filename/filename.laz"
# output_las = "path/to/full/filename/filename.laz"
# HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
# subprocess.run(HAG_cmd)

**Multiple Files**

In [ ]:
# # multiple files
# pathname = "path/to/lid/folder/"
# output_pathname = "path/to/lid/folder/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     HAG_cmd = ['pdal', 'translate',input_las, output_las, '--json', HAG_json]
#     subprocess.run(HAG_cmd)

**Parallel Processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         executor.map(lidar_functions.HAG_dem, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

Applied

In [ ]:
# parallel processing
# time = 1.1 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path  + s for s in onlyfiles]
        executor.map(lidar_functions.HAG_dem, full_path, output_path_full)

In [ ]:
(toc-tic)/60

## Add a new dimension
Add a new dimension to the file but maintain Z values

In [ ]:
# # this works to add a new dimension
# output_las_hag = 'input/lid/files/lidar.laz'
# filter_dem = '--filters.hag_dem.raster=' + target_DEM
# hag_addDim = ['pdal', 'translate', input_las_hag, output_las_hag, 'hag_dem' ,filter_dem, '--writers.las.extra_dims=HeightAboveGround=float32']
# subprocess.run(hag_addDim)

# Vertical Bias Correction

## Move files
Make a copy of road covered tiles. <br>
The spatial extents of the road polygon are 738382,4371452 : 738686,4372063 <br>
chose only files with a min x > 737000 and a min y > 4370000

In [ ]:
# # defining source and destination
# # paths
# src = 'path/to/source/folder/*.la*'
# trg = 'path/to/target/folder/target'
# file_paths = glob.glob(src)

# for files in file_paths:
#     filename = os.path.basename(files)
#     if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
#     # copying the files to the
#     # destination directory
#         shutil.copy2(files, trg)

Applied

In [ ]:
# defining source and destination
# paths
src = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/HAG/*.la*'
trg = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_lid'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

In [ ]:
# defining source and destination
# paths
src = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/HAG/*.la*'
trg = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/hwy89_vertical_bias/target_lid'
file_paths = glob.glob(src)

for files in file_paths:
    filename = os.path.basename(files)
    if int(files[-18:-12]) > 737000 and int(files[-11:-4]) > 4370000:
    # copying the files to the
    # destination directory
        shutil.copy2(files, trg)

In [ ]:
int(os.path.basename(file_paths[0])[-11:-4])

## Merge
Merge files over the road

In [ ]:
# input_path = 'path/to/target_lid/*.la*' # define path of input files
# output_fname = 'path/to/target/filename.laz'# set output filename
# input_fname = glob.glob(input_path) # save to list
# pdal_merge_command = input_fname
# pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
# #pdal_merge_command.insert(0,'-f')
# pdal_merge_command.insert(0,'merge')
# pdal_merge_command.insert(0,'pdal')
# subprocess.run(pdal_merge_command)

Applied

In [ ]:
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_lid/*.la*' # define path of input files
input_fname = glob.glob(input_path) # save to list
output_fname = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/target_lid/ASO_SCB_20160417_hwy89_merge.'+input_fname[0][-3:]# set output filename
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

In [ ]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_lid/*.la*' # define path of input files
input_fname = glob.glob(input_path) # save to list
output_fname = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_lid/NCALM_SCB_2014_hwy89_merge.'+input_fname[0][-3:]# set output filename
pdal_merge_command = input_fname
pdal_merge_command.insert(len(pdal_merge_command),output_fname) # insert output file to list
#pdal_merge_command.insert(0,'-f')
pdal_merge_command.insert(0,'merge')
pdal_merge_command.insert(0,'pdal')
subprocess.run(pdal_merge_command)

## Clip
Crop file to outline of the road shapefile

In [ ]:
# name JSON file
clip_json = json_base_path +'clip_to_geometries.json'

In [ ]:
path_to_shapefile = 'SCB/supporting_files/bounding_box/hwy89_poly.shp'

In [ ]:
# extract wkt from hwy 89 polygon
hwy89 = shapefile.Reader(path_to_shapefile)
geom=[]
for s in hwy89.shapes():
    geom.append(pygeoif.geometry.as_shape(s)) 
poly_base = pygeoif.MultiPolygon(geom)

In [ ]:
# create a pipeline and save to a json file 
filter_crop = {'type':'filters.crop',
                 'polygon':poly_base.wkt}
pipeline_list = [filter_crop]
pipeline_dict = {'pipeline' : pipeline_list}
with open(clip_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', clip_json]
# subprocess.run(pdal_commands)

Applied

In [ ]:
input_las = '/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/target_lid/NCALM_SCB_2014_hwy89_merge.las'
output_las = '/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.las'

In [ ]:
pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', clip_json]
subprocess.run(pdal_commands)

## Calculate Stats Over Control Area
In Kostadinov et al., 2019 the vertical bias was corrected using the lowest 10th percentile value between the snow on and snow off flights above the road. We will calculate a number of statistics

### Calculate Stats

In [ ]:
# input_las - HAG, las file clipped to the road
# output_path - path to output files
# base_name - string, typically flight name
def calculate_vertical_bias(input_las):
    # convert height only to txt file
    output_las_txt = input_las[:-3]+'csv'
    txt_cmd = ['pdal', 'translate', input_las, output_las_txt, '-w', 'writers.text', '--writers.text.format=csv','--writers.text.order=Z', '--writers.text.keep_unspecified=false']
    subprocess.run(txt_cmd)
    # calculate stats
    hag_arr = np.loadtxt(output_las_txt,skiprows=1)
    lowest_10th_per = np.nanpercentile(hag_arr, 10)
    mean_hag = np.nanmean(hag_arr)
    median_hag = np.nanmedian(hag_arr)
    stats = ["lowest_10th",lowest_10th_per, "mean",mean_hag, "median", median_hag]
    return(stats)

Applied
Using 2014 NCALM Flight

In [ ]:
ASO_20160326_hwy89_stats = calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/hwy89_vertical_bias/clipped/ASO_SCB_20160326_hwy89_clip.laz')
ASO_20160417_hwy89_stats = calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/hwy89_vertical_bias/clipped/ASO_SCB_20160417_hwy89_clip.laz')
ASO_20160518_hwy89_stats = calculate_vertical_bias('SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/hwy89_vertical_bias/clipped/ASO_SCB_20160518_hwy89_clip.las')

In [ ]:
NCALM_2014_hwy89_stats = calculate_vertical_bias('SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/hwy89_vertical_bias/clipped/NCALM_SCB_2014_hwy89_clip.las')

In [ ]:
print(ASO_20160326_hwy89_stats)
print(ASO_20160417_hwy89_stats)
print(ASO_20160518_hwy89_stats)
print(NCALM_2014_hwy89_stats)

## Correct Z Values
Using PDAL filters.assign we can add a value to each lidar point
See lidar_functions.py - correct_by-target_val <br>

### Correct LAS Only
This involves simple command line function operations instead of a json derived pipeline. 

In [ ]:
# input_lid = 'path/to/input/filename/filename.laz'
# output_lid = 'path/to/output/file/filename.laz'
# filters_assign = '--filters.assign.value=Z=Z'+target_val
# assign_cmd = ['pdal', 'translate', input_lid, output_lid, 'assign' ,filters_assign]
# subprocess.run(assign_cmd)

In [ ]:
# # or use function
# # assign target val
# correct_by_targetVal_pipeline(target_val)
# full_input_path = 'path/to/input/filename/filename.laz'
# full_output_path = 'path/to/output/file/filename.laz'
# target_val = 0
# lidar_functions.correct_by_targetVal(full_input_path, full_output_path, target_val)

**Parallelize**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         target_vals = np.repeat(target_val, len(full_path))
#         executor.map(lidar_functions.correct_by_targetVal, full_path, output_path_full, target_vals) 
# toc = time.perf_counter()

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/corrected_las/'
target_val = 'Z+'+str(abs(NCALM_2014_hwy89_stats[1])) # lowest 10th percentile 
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path + s for s in onlyfiles]
        target_vals = np.repeat(target_val, len(full_path))
        executor.map(lidar_functions.correct_by_targetVal, full_path, output_path_full, target_vals)
toc = time.perf_counter()

In [ ]:
lidar_functions.correct_by_targetVal(full_path[0], output_path_full[0], target_vals[0])

In [ ]:
lidar_functions.correct_by_targetVal(full_path[0], output_path_full[0], target_val)

In [ ]:
filters_assign = '--filters.assign.value=Z=Z+0.08'
assign_cmd = ['pdal', 'translate', full_path[0], output_path_full[0], 'assign' ,filters_assign]
subprocess.run(assign_cmd)

### Correct LAS and Rasterize
For this section, we overwrite the json for each flight depending on the target value. This is beneficial for saving memory (no redundant corrected las files created)

In [ ]:
correct_rasterize_json = json_base_path + 'correct_by_targetVal_rasterize.json'

In [ ]:
# note that the corrected Z is Z - target val, because we are using the height above ground
# input: target_assign - str e.g 'Z+0.08'
def correct_by_targetVal_pipeline(target_assign):
    readers_las = {'type':'readers.las'}
    filters_assign = {'type': 'filters.assign',
                      'value':"Z="+target_assign}
    writers_gdal= {"type": "writers.gdal",
                  'output_type': 'mean',
                  'resolution': '1.0',
                  'radius': '0.7'}
    pipeline_list = [readers_las,filters_assign,writers_gdal]
    pipeline_dict = {'pipeline' : pipeline_list}
    with open(correct_rasterize_json, 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

**Single Files**

In [ ]:
# full_input_path = 'path/to/input/lid/filename.las'
# full_output_path = 'path/to/output/lid/filename.las'
# target_val = -9999
# correct_by_targetVal_pipeline(target_val)
# lidar_functions.correct_by_targetVal_rasterize(full_input_path, full_output_path)

Applied

**Parallel Processing**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + '/' + s for s in onlyfiles]
#         output_path_full = [output_path + '/' + s for s in onlyfiles]
#         executor.map(lidar_functions.correct_by_targetVal, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

Applied

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/HAG/'
output_path = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/'
target_val = ASO_20160518_hwy89_stats[1] # lowest 10th percentile 
correct_by_targetVal_pipeline(target_val)
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path + s[:-3] +'tif' for s in onlyfiles]
        executor.map(lidar_functions.correct_by_targetVal_rasterize, full_path, output_path_full)
toc = time.perf_counter()

# Vegetation Classifications

## Filter By Veg Strata

### [-0.15,0.15)

In [ ]:
range_json_neg0pt15_0pt15 = json_base_path+'filter_pts_neg0pt15_0pt15.json'
reader_dict = {'type':'readers.las'}
filter_range_neg0pt15_0pt15 = {"type":"filters.range",
                "limits":"Z[-0.15:0.15)"}
writers_gdal_count= {"type": "writers.gdal",
              'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7',
              'window_size':2}
pipeline_list = [reader_dict,filter_range_neg0pt15_0pt15,writers_gdal_count]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json_neg0pt15_0pt15, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

### [0.15,2)

In [ ]:
range_json_0pt15_2 = json_base_path+'filter_pts_0pt15_2.json'
reader_dict = {'type':'readers.las'}
filter_range_0pt15_2 = {"type":"filters.range",
                "limits":"Z[0.15:2)"}
writers_gdal_count= {"type": "writers.gdal",
              'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7'}
pipeline_list = [reader_dict,filter_range_0pt15_2,writers_gdal_count]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json_0pt15_2, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

### [2,inf)

In [ ]:
range_json_2 = json_base_path+'filter_pts_2.json'
reader_dict = {'type':'readers.las'}
filter_range_2 = {"type":"filters.range",
                "limits":"Z[2:)"}
writers_gdal_count= {"type": "writers.gdal",
              'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7'}
pipeline_list = [reader_dict,filter_range_2,writers_gdal_count]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json_2, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

### [2,inf) Ground

In [ ]:
range_json_2_ground = json_base_path+'filter_pts_2_ground.json'
reader_dict = {'type':'readers.las'}
filter_range_2_ground = {"type":"filters.range",
                "limits":"Z[2:), Classification[2:2]"}
writers_gdal= {"type": "writers.gdal",
              'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7'}
pipeline_list = [reader_dict,filter_range_2_ground,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json_2_ground, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

### [2,inf) Nonground

In [ ]:
range_json_2_nonground = json_base_path+'filter_pts_2_nonground.json'
reader_dict = {'type':'readers.las'}
filter_range_2_nonground = {"type":"filters.range",
                "limits":"Z[2:), Classification![2:2]"}
writers_gdal= {"type": "writers.gdal",
              'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7'}
pipeline_list = [reader_dict,filter_range_2_nonground,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
with open(range_json_2_nonground, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

**Single File**

In [ ]:
# input_las = 'path/to/input/lid/filename.las'
# output_las = 'path/to/output/lid/filename.las'
# strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
# subprocess.run(strata_cmd)

**All Files in Directory**

In [ ]:
# tic = time.perf_counter()
# pathname = "path/to/lid/folder/"
# output_pathname = 'path/to/lid/folder/'
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file
#     strata_cmd = ['pdal', 'translate', input_las,  output_las, '--json',range_json]
#     subprocess.run(strata_cmd)
# toc = time.perf_counter()

**Parallelization**

In [ ]:
# tic = time.perf_counter()
# input_path = 'path/to/lid/folder/'
# output_path = 'path/to/lid/folder/'
# if __name__ == '__main__':
#     with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
#         onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
#         full_path = [input_path + s for s in onlyfiles]
#         output_path_full = [output_path + s for s in onlyfiles]
#         executor.map(lidar_functions.range_json, full_path, output_path_full) #running 10 times
# toc = time.perf_counter()

#### Combined Pipeline Method

In [ ]:
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_neg0pt15_0pt15.json'

input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_classes/vegStrat_neg0pt15_0pt15.tif'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
input_list = [input_path + s for s in onlyfiles]

In [ ]:
filename_dict = {}
tags = ['']*len(input_list)
filenames = ['']*len(input_list)*2
for i in range(len(input_list)):
    filename_dict['filename_'+str(i)] = {'type':'readers.las','filename':input_list[i]}
    filename_dict['filename_'+str(i)+'b'] = {"type":"filters.range","limits":"Z[-0.15,0.15)", 'tag':'A_'+str(i)}
    tags[i] = 'A_'+str(i)
for j in range(len(filename_dict)):
    filenames[j] = filename_dict[list(filename_dict)[j]]    

In [ ]:
# merge all las files or stages
F1 = {'type':'readers.las','filename':input_list[0]}
R1 = {"type":"filters.range","limits":"Z[-0.15,0.15)", 'tag':'A_0'}
F2 = {'type':'readers.las','filename':input_list[1]}
R2 = {"type":"filters.range","limits":"Z[-0.15,0.15)", 'tag':'A_1'}

filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": ['A_0','A_1']}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'count',
              'resolution': '1.0',
              'radius': '0.7',
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
# pipeline_list = filenames.copy()
# pipeline_list.append(filter_merge)
# pipeline_list.append(writers_gdal)
pipeline_list = [F1,R1,F2,R2,filter_merge,writers_gdal]
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
tic = time.perf_counter()
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)
toc = time.perf_counter()

## Move to raster workflow

test script

In [ ]:
# # name JSON file
# rasterize_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json'
# # input_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000.las' # define input las full file path
# # output_las = 'lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000_GFtutorial.las' # define input las full file path

# # create a pipeline and save to a json file 
# reader_dict = {'type':'readers.las'}
# filter_gdal= {"type": "writers.gdal",
#               'gdaldriver':'GTiff',
#               'output_type': 'mean',
#               'resolution': 1.0}
# #output_dtm = "lidar_processing/python_scripts/PDAL/test_file/ncalm_2014_732000_4373000_DTMtutorial.tif"


# pipeline_list = [reader_dict,filter_gdal]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(rasterize_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)

In [ ]:
# See below for better solution

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--writers.gdal.resolution=1','--writers.gdal.output_type=mean']
#     subprocess.run(pdal_commands)

In [ ]:
# didn't work because the pipeline wants an input/output

# pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/NCALM/"
# output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/NCALM/"
# onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
# for file in onlyfiles:
#     input_las = pathname + file
#     output_las = output_pathname + file[:-4]+".tif"
#     pdal_commands = ['pdal', 'translate', input_las, output_las, '--json', rasterize_json]
#     subprocess.run(pdal_commands)

In [ ]:
# test new formatting
# input_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# output_las = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_000.tif'
# reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160417/mcc_part_b_tile_004_000.las'
# rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]

In [ ]:
# test on ASO data
# for i in [0, 1, 2, 3, 4, 5]:
#     writer = '--writers.gdal.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
#     reader = '--readers.las.filename=lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.las'
#     rasterize_command = ['pdal', 'pipeline', 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_tif.json', writer, reader]
#     subprocess.run(rasterize_command)

### Clipping Geometries
The goal of this portion of the code is to clip the raster based on the control areas (in the case of SCB, hwy 89)<br>
See: https://pdal.io/tutorial/clipping/index.html#clipping

In [ ]:
[
    "autzen.laz",
    {
      "type":"filters.overlay",
      "dimension":"Classification",
      "datasource":"attributes.vrt",
      "layer":"OGRGeoJSON",
      "column":"CLS"
    },
    {
      "type":"filters.range",
      "limits":"Classification[5:5]"
    },
    "output.las"
]

In [ ]:
# name JSON file
clipping_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/clip_las_to_shp.json'

filter_overlay_dict = {"type":"filters.overlay",
                       "dimension":"Classification",
                       "datasource":"SCB/bounding_box/hwy89_poly.shp",
                       "column":"OBJECTID"}
filter_range_dict = {"type":"filters.range",
                     "limits":"Classification[4193:4193]"}

pipeline_list = [filter_overlay_dict,filter_range_dict]
with open(clipping_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
pathname = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/reproject/20160518_toNAD83/toNAVD88/'
output_pathname = "lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/ground_filtered/ASO_20160518/reproj_NAD83_NAVD88/"
onlyfiles = [f for f in os.listdir(pathname) if os.path.isfile(os.path.join(pathname, f))]
pdal_clip = ['pdal', 'translate', input_las, output_las, '--json', output_json]
subprocess.run(pdal_clip)

# Applied

## Pre-Processing

### Info

In [ ]:
# define input file
input_lid = r'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/CA_20210429_pN94S_f1_tiledlas_fl195341_ch1_ti-6720_tj-4500_ts60_v2_UTMZ11.laz'
pdal_info_cmd = ['pdal','info',input_lid] # general info
pdal_metadata_cmd = ['pdal','info',input_lid,'--metadata'] # full file metadata, including details crs
subprocess.run(pdal_metadata_cmd)

In [ ]:
pdal_info_results = subprocess.run(pdal_metadata_cmd, stdout = subprocess.PIPE) # stout (standard out), PIPE indicates that a new pipe to the child should be created
pdal_info_dict = json.loads(pdal_info_results.stdout.decode()) # create dict with metadata info
#pdal_info_dict # print results

In [ ]:
# pdal_info_dict

### Retile

In [ ]:
lidar_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
retile_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/retile_uo/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
for files in onlyfiles:
    full_path = os.path.join(lidar_folder, files)
    output_path = retile_folder+'#' + files
    retile_command = ['pdal', 'tile', full_path, output_path, '--length=1000','--buffer=50']
    subprocess.run(retile_command)
    

### Rename

In [ ]:
pth = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/retile_uo/730000_4364000.las'
full_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/retile_uo/21_730000_4364000.las'
if os.path.exists(pth):
    lidar_folder = os.path.dirname(pth)
    onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
    full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
    full_str = ','.join(full_paths)
    num_occurences = full_str.count(pth[:-4])
    new_name_b = pth[:-4]+'_'+str(num_occurences)+pth[-4:]
    os.rename(full_path, new_name_b)

In [ ]:
lidar_folder = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/retile_uo/'
tic = time.perf_counter()
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
        onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
        full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
        executor.map(lidar_functions.rename_llx_lly_b, full_path) #running 10 times
toc = time.perf_counter()

### Save Tile Boundaries

In [ ]:
# one folder
tic = time.perf_counter()
lidar_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/'
output_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/tindex/original/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
        full_path = [os.path.join(lidar_folder, s) for s in onlyfiles]
        # change depending on directory formats
        #output_path = [os.path.join(os.path.basename(lidar_folder),'tindex/tiles/' + s[:-3] + 'sqlite') for s in onlyfiles]
        output_path = [output_folder + s[:-3] + 'sqlite' for s in onlyfiles]
        executor.map(lidar_functions.create_tindex, full_path, output_path) #running 10 times
toc = time.perf_counter()

### Copy Files

In [ ]:
tic = time.perf_counter()
lidar_folder = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/laz/'
output_path = 'MRB/Merced_lidar/ASO/ASO_MRB_20210429/ICB_tiles/'
onlyfiles = [f for f in os.listdir(lidar_folder) if os.path.isfile(os.path.join(lidar_folder, f))]
full_paths = [os.path.join(lidar_folder, s) for s in onlyfiles] 
output_paths = np.repeat(output_path, len(full_paths))
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        executor.map(lidar_functions.copy_lid_by_ext_ICB, full_paths, output_paths)
toc = time.perf_counter()

## Snow-Off Processing

### Create DEM

In [ ]:
DEM_json = json_base_path+'DEM_from_las.json' # define path to json files

In [ ]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/testc_raw.tif'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
input_list = [input_path + s for s in onlyfiles]

In [ ]:
filename_dict = {}
tags = ['']*len(input_list)
filenames = ['']*len(input_list)
for i in range(len(input_list)):
    filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
    tags[i] = 'A_'+str(i)
    filenames[i] = filename_dict[list(filename_dict)[i]]

In [ ]:
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7',
               'window_size':3,
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(DEM_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
tic = time.perf_counter()
pdal_cmd = ['pdal','pipeline', DEM_json]
subprocess.run(pdal_cmd)
toc = time.perf_counter()

### Heigh Above Ground

In [ ]:
# parallel processing
# time = 1.1 min
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/NAD83_NAD83_epoch2010/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path  + s for s in onlyfiles]
        executor.map(lidar_functions.HAG_dem, full_path, output_path_full)

### Vegetation Strata

**Parallelization Method**

In [ ]:
tic = time.perf_counter()
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/vegStrat_neg0pt15_0pt15/tif/'
if __name__ == '__main__':
    with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
        onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
        full_path = [input_path + s for s in onlyfiles]
        output_path_full = [output_path + s[:-3] + 'tif' for s in onlyfiles]
        executor.map(lidar_functions.filter_pts_neg0pt15_0pt15, full_path, output_path_full) 
toc = time.perf_counter()

**Combined Pipeline Method**

In [ ]:
filterMergeRasterize_json = json_base_path+'filterMergeRasterize_neg0pt15_0pt15.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_0pt15_2.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_2.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_2_ground.json'
# filterMergeRasterize_json = json_base_path+'filterMergeRasterize_2_nonground.json'

In [ ]:
input_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/HAG/'
output_tif = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_classes/vegStrat_neg0pt15_0pt15.tif'
onlyfiles = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
input_list = [input_path + s for s in onlyfiles]

In [ ]:
filename_dict = {}
tags = ['']*len(input_list)
filenames = ['']*len(input_list)
for i in range(len(input_list)):
    filename_dict['filename_'+str(i)] = {'filename':input_list[i], 'tag':'A_'+str(i)}
    tags[i] = 'A_'+str(i)
    filenames[i] = filename_dict[list(filename_dict)[i]]

In [ ]:
# filter out the ground points of the tiles
filter_range = {"type":"filters.range",
                "limits":"Classification[2:2]"}
# merge all las files or stages
filter_merge = {"type":"filters.merge",
               "tag": "merged",
               "inputs": tags}
# write merged las to raster
writers_gdal= {"type": "writers.gdal",
               'output_type': 'mean',
              'resolution': '1.0',
              'radius': '0.7',
               'window_size':3,
               'filename': output_tif}
# Append each stage to a list prior to saving to json 
pipeline_list = filenames.copy()
pipeline_list.append(filter_merge)
#pipeline_list.append(filter_range)
pipeline_list.append(writers_gdal)
pipeline_dict = {'pipeline' : pipeline_list}
# save to json
with open(filterMergeRasterize_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)

In [ ]:
tic = time.perf_counter()
pdal_cmd = ['pdal','pipeline', filterMergeRasterize_json]
subprocess.run(pdal_cmd)
toc = time.perf_counter()

## ____________________________

# Extras

## File format conversions

### Convert .las to .txt
see: https://pdal.io/stages/writers.text.html

In [ ]:
# set up json file commands
output_txt = '/Volumes/cpiske/lidar_processing/python_scripts/PDAL/test_las/mcc_part_b_tile_004_000Test.asc'
output_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/las_to_txt.json'

# create a pipeline and save to a json file 

filter_dict = {'type':'readers.las',
               'override_srs': "EPSG:4326",
              'filename': input_las} # we are reading in a las file
rasterize_dict = {'type':'writers.las',
'format':'geojson',
'order':'X,Y,Z',
'keep_unspecified':'false',
'filename':output_txt}


pipeline_list = [filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
json_path = 'lidar_processing/python_scripts/PDAL/JSON/las_to_txt.json'
pdal_commands = ['pdal', 'pipeline', json_path]
subprocess.run(pdal_commands)

### .laz to .las

In [ ]:
# # set up json file commands
# input_laz = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.laz'
# output_las = 'lidar_processing/python_scripts/PDAL/test_las/ASO_USCAMB20180425f1a1_180425_1_dem_filter.las'
# output_json = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'

# # create a pipeline and save to a json file 

# filter_dict = {'type':'readers.las',
#                'filename': input_las} # we are reading in a las file
# translate_dict = {'type':'writers.las',
#                   "a_srs": "EPSG:4326",
#                   'filename':output_las}


# pipeline_list = [filter_dict, translate_dict]
# pipeline_dict = {'pipeline' : pipeline_list}
# with open(output_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
# json_path = 'lidar_processing/python_scripts/PDAL/JSON/laz_to_las.json'
# pdal_commands = ['pdal', 'pipeline', json_path]
# subprocess.run(pdal_commands)

# Raster Caluclations

In [ ]:
for i in [0,1,2,3,4,5]:
    apr_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160417/mcc_part_b_tile_004_00'+str(i)+'.tif'
    may_elev = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/rasterize/ASO_20160518/mcc_part_b_tile_004_00'+str(i)+'.tif'
    output = 'lidar_processing/PDAL_testFiles_tutorials/test_las/SCB/raster_subtract/mcc_part_b_tile_004_00'+str(i)+'.tif'
    raster_sub = ['gdal_calc.py', '-a', apr_elev, '-b', may_elev, '--calc="a - b"', '--outfile', output]
    subprocess.run(raster_sub)

# Theo Code

In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

z_min = 0.15
z_max = 2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.tif'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_tif.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'GTiff',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


In [ ]:
input_las = 'SCB/kost_lidar_data/ASO_2016/2016_05_18/WGS84_G1762_to_NAD83_NAVD88/mcc_part_b_tile_004_000.las'

# z_min = 0.15
# z_max = 2
# z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
output_raster = 'lidar_processing/python_scripts/PDAL/test_file/mcc_part_b_tile_004_000.asc'
output_json = 'lidar_processing/python_scripts/PDAL/JSON/las_to_asc.json'
resolution = 0.01

filter_dict = {'type':'filters.range', 'limits':z_range}
rasterize_dict = {'filename':output_raster,
'gdaldriver':'XYZ',
'output_type':'count',
'resolution':resolution,
'type': 'writers.gdal'}


pipeline_list = [input_las, filter_dict, rasterize_dict]
pipeline_dict = {'pipeline' : pipeline_list}
with open(output_json, 'w') as out:
    json.dump(pipeline_dict, out, indent=4)


sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz

where path_to_laz_folder is the path to the LAS/LAZ file (you just need the folder path, not the file path).

:/input is the new folder that will be created in your Docker container that will hold your point cloud.

0b is just the image id of pdal

/input/test.laz is the path to the point cloud in the Docker container.


In [ ]:
path_to_laz_folder = 'lidar_processing/python_scripts/PDAL/test_las'


In [ ]:
sudo docker run -v path_to_laz_folder:/input 0b pdal info /input/test.laz


In [ ]:
import json
import subprocess
import pdal

def assemblePipeline(input_las, list_of_dicts):
    pipeline_list = [input_las]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline' : pipeline_list}
    return pipeline_dict

def makeHeightFilter(height, buffer):
    z_min = height - buffer/2
    z_max = height + buffer/2
    z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
    heightDict = {'type':'filters.range', 'limits':z_range}
    return heightDict

def makeRasterizeFilter(output_raster, resolution, epsg):
    rasterize_dict = {'filename':output_raster,
                      'gdaldriver':'GTiff',
                      'output_type':'count',
                      'resolution':resolution,
                      'override_srs' : epsg,
                      'type': 'writers.gdal'}
    return rasterize_dict

def convertTifForPIL(input_raster, output_raster, epsg):
    ''' GDAL bindings are an alien concept to me, so I gave up and used
        subprocess.'''
    commands = ['gdal_translate', input_raster, output_raster, '-ot', 'Byte', '-a_srs', epsg]
    subprocess.run(commands)


def buildHeightSlice(input_las, height, buffer, output_raster, resolution, epsg, json_path=None):
    filter_dict = makeHeightFilter(height, buffer)
    rasterize_dict = makeRasterizeFilter(output_raster, resolution, epsg)
    filter_list = [filter_dict, rasterize_dict]
    pipeline_dict = assemblePipeline(input_las, filter_list)

    if json_path is not None:
        with open(json_path, 'w') as out:
            json.dump(pipeline_dict, out, indent=4)
        pdal_commands = ['pdal', 'pipeline', json_path]
        subprocess.run(pdal_commands)
    else:
        pdal_commands = json.dumps(pipeline_dict)
        pipeline = pdal.Pipeline(pdal_commands)
        pipeline.execute()

input_las = '/Users/theo/data/las/TLS_0244_20180612_01_v003_30m_clip_height_norm.las'
height = 1.37
buffer = 0.05
z_min = height - buffer/2
z_max = height + buffer/2
z_range = 'Z[' + str(z_min) + ':' + str(z_max) + ']'
temp_raster = '/Users/theo/Pictures/almost_cool.tif'
final_raster = '/Users/theo/Pictures/cool.tif'
resolution = 0.01
epsg = 'EPSG:3310'

buildHeightSlice(input_las, height, buffer, temp_raster, resolution, epsg)
convertTifForPIL(temp_raster, final_raster, epsg)

In [ ]:
import json
import subprocess
import pdal
import argparse

# Create flags for the user to utilize.
parser = argparse.ArgumentParser(description="Generate JSON pipeline to generate DTM from a point cloud.")
      
required = parser.add_argument_group('Required arguments')
required.add_argument('-crs', '--coordinate_system', required=True, action='store', help="EPSG code.")
required.add_argument('-i', '--infile', required=True, action='store', help="Input path to point cloud")
required.add_argument('-o', '--outfile', required=True, action='store', help="Output path.")
args = parser.parse_args()

def generateJSON(infile, list_of_dicts):
    pipeline_list = [infile]
    pipeline_list.extend(list_of_dicts)
    pipeline_dict = {'pipeline': pipeline_list}
    with open("pipeline.json", 'w') as out:
        json.dump(pipeline_dict, out, indent=4)

def generateDTM(epsg, infile, outfile):
    reproject_dict = {"type": "filters.reprojection",
                      "out_srs": "EPSG:{}".format(epsg)}
    reclassify_zero_dict = {"type": "filters.assign",
                       "assignment": "Classification[:]=0"}
    elm_dict = {"type": "filters.elm"}
    outlier_dict = {"type": "filters.outlier"}
    smrf_dict = {"type": "filters.smrf", "ignore": "Classification[7:7]",
                 "slope": 0.2, "window": 16, "threshold": 0.45, "scalar": 1.2}
    range_dict = {"type":"filters.range", "limits":"Classification[2:2]"}
    output_dict = {"filename": outfile, "gdaldriver": "GTiff", "output_type": "all", "resolution": 0.01, "type": "writers.gdal"}
    list_of_dicts = list([reproject_dict, reclassify_zero_dict, elm_dict, outlier_dict, smrf_dict, range_dict, output_dict])
    generateJSON(infile, list_of_dicts)
    pdal_cmds = ['pdal', 'pipeline', 'pipeline.json']
    subprocess.run(pdal_cmds)
    
generateDTM(args.coordinate_system, args.infile, args.outfile)

# More helpful things

### Get stats of a dataset
see: https://www.spatialised.net/lidar-qa-with-pdal-part-1/

In [ ]:
# name JSON file

stats_json = 'lidar_processing/python_scripts/PDAL_workflow/JSON/stats.json'

In [ ]:
reader_las = {"type":"readers.las",
              "filename": input_las_stats}
filter_stats = {"type":"filters.stats",
                "dimensions":"Z",
                "global":"Z",
                "advanced":"true"}
pipeline_list = [reader_las, filter_stats]
#pipeline_dict = {reader_las, filter_stats}
# with open(stats_json, 'w') as out:
#     json.dump(pipeline_dict, out, indent=4)


In [ ]:
pipeline_list

In [ ]:
pipeline = pdal.Pipeline(json.dumps(pipeline_list))
pipeline.execute()

In [ ]:
json.loads(pipeline.metadata)["metadata"]["filters.stats"]["statistic"]

## Navigating folders/files

**create list of files/folders with a wildcard (*)**

In [ ]:
# ex. list all files in folder4 that end in .laz = folder1/folder2/folder3/folder4/*.laz
# ex. list all folders named folder3 in folder 1 = foler1/*/folder3
# ex. list all list all contents in folder2 = folder2/* - note just folder 2, no subdirectories
glob_cmd = 'path'
glob_exe = glob.glob(file_glob_cmd)

**create a list with all directories/subdirectories on a path**

In [ ]:
all_folders = [x[0] for x in os.walk('path')]

**create a list with all directories/subdirectories on a path with specific folders**

In [ ]:
all_folders = [x[0] for x in os.walk('path')]
index_pos_list = [ i for i in range(len(all_folders)) if all_folders[i][-6:] == 'retile' ] # must change this to meet requirements
full_list = [all_folders[i] for i in index_pos_list]

**get name of the directory just above one listed**

In [ ]:
subdirname = os.path.basename(os.path.dirname('path'))

**create a list with only filenames**

In [ ]:
onlyfiles = [f for f in os.listdir('path') if os.path.isfile(os.path.join('path', f))]

**create a list with full file paths**

In [ ]:
full_path = ['path' + '/' + s for s in onlyfiles]

## Parallel Processing

In [ ]:
# time = 142.5
tic = time.perf_counter()
list(map(function, args));
toc = time.perf_counter()

In [ ]:
# time = 53 s
tic = time.perf_counter()
if __name__ == "__main__":
    pool = Pool(3)
    pool.map(function, arg)
    pool.close()
toc = time.perf_counter()

In [ ]:
# time = very fast? .06s
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()

In [ ]:
# time = 0.22
tic = time.perf_counter()
if __name__ == "__main__":
    executor = concurrent.futures.ProcessPoolExecutor(max_workers=3)
    executor.map(function, arg)
toc = time.perf_counter()